<a href="https://colab.research.google.com/github/akhilgupta1093/JaneAustenGenerator/blob/master/JaneAustenGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# All import statements
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

Using TensorFlow backend.


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive"

In [0]:
# Load text and convert to lowercase
filename = "/content/drive/My Drive/prideprejudice.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [9]:
# create mapping of unique chars to ints
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total chars:", n_chars)
print("Total vocab:", n_vocab)

Total chars: 756216
Total vocab: 56


In [10]:
# Prepare the dataset
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = raw_text[i:i + seq_length]
  seq_out = raw_text[i + seq_length]
  dataX.append([char_to_int[char] for char in seq_in])
  dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns:", n_patterns)

Total Patterns: 756116


In [0]:
# Reshape X to be [samples, time steps, features] as is needed for LSTM
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize
X = X / float(n_vocab)
# One hot encode the output variable
y = np_utils.to_categorical(dataY)

In [0]:
# Define the model
model = Sequential()
model.add( LSTM(256, input_shape=(X.shape[1], X.shape[2])) )
model.add( Dropout(0.2) )
model.add( Dense(y.shape[1], activation='softmax') )

In [0]:
# Compile the model
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [0]:
# Checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [16]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
756116/756116 [==============================] - 893s 1ms/step - loss: 2.5805

Epoch 00001: loss improved from inf to 2.58047, saving model to weights-improvement-01-2.5805.hdf5
Epoch 2/20
756116/756116 [==============================] - 885s 1ms/step - loss: 2.3771

Epoch 00002: loss improved from 2.58047 to 2.37708, saving model to weights-improvement-02-2.3771.hdf5
Epoch 3/20
756116/756116 [==============================] - 883s 1ms/step - loss: 2.2507

Epoch 00003: loss improved from 2.37708 to 2.25075, saving model to weights-improvement-03-2.2507.hdf5
Epoch 4/20
756116/756116 [==============================] - 878s 1ms/step - loss: 2.1464

Epoch 00004: loss improved from 2.25075 to 2.14644, saving model to weights-improvement-04-2.1464.hdf5
Epoch 5/20
756116/756116 [==============================] - 877s 1ms/step - loss: 2.0620

Epoch 00005: loss improved from 2.14644 to 2.06201, saving model to weights-improvement-05-2.0620.hdf5
Epoch 6/20
756116/756116 [=============

In [0]:
# Load the network weights
filename = "weights-improvement-20-1.6592.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
# Make reverse mapping
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [26]:
# Pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:
" 

      the colour which had been driven from her face, returned for half
      a minute with an add "


In [27]:
# Generate characters
import sys
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

ersence of the sooe the same tooe the was to
      see mote than the sooe of the sooe the sase of the sooe thet
      seruind to the person the same tooe the same tooe the was of
      seeeng the same tooe the same tooe the same tooe the was oo
      sereruly oo the sooe the sare of the sooe the same tooe the
      consersance of the poom of the sooe the same tooe the was to
      shete that her faal of the sooe whse to be iose the seme of the
      consersanion of the sooe the same tooe the same tooe the same
      connestion of the poom of the sooe the sase of the sooe thet
      sereruly to the person the sase of the sooe the same tooe the
      consersanco of the sooe the sase of the sooe the same tooe the
      consersanco of the sooe the sase of the sooe the same tooe the
      consersanco of the sooe the sase of the sooe the same tooe the
      consersanco of the sooe the sase of the sooe the same tooe the
      consersanco of the sooe the sase of the sooe the same tooe the
    